In [1]:
import os
os.chdir("../../")
import pandas as pd
import numpy as np
import spacy
from tqdm import tqdm

In [25]:
nlp = spacy.load("en_core_web_lg")

def process_data(filename, folderpath):
    df = pd.read_csv(folderpath + filename).drop("Unnamed: 0", axis=1)
    df["news"] = df["news"].replace("\n", "")
    df["date"] = pd.to_datetime(df["date"], errors="coerce")
    df["ym"] = [str(d.year) + "-" + str(d.month) for d in df.date]
    return df


def extract_entities(corpus: str):
    doc = nlp(corpus)
    ner_labels = ["LOC", "ORG", "GPE", "WORK_OF_ART"]
    ner_dict = {"ner": ""}
    for e in doc.ents:
        if e.label_ in ner_labels and e.text not in ner_dict["ner"]:
            ner_dict["ner"] += e.text + ", "

    ner_dict["ner"] =  ner_dict["ner"][:-2]
    return ner_dict

In [31]:
parent_dir = os.getcwd() + "/data/text"
country_dirs = [
    f"{parent_dir}/{file}/" for file in os.listdir(parent_dir)
    if file != ".DS_Store"
]

for country_dir in country_dirs[1:]:
    files = [
        file for file in os.listdir(country_dir)
        if "news" in file and "ner" not in file
    ]
    for file in files:
        df = process_data(file, country_dir)
        name = file.replace(".csv", "") + "_ner.csv"
    
        output = []
        with tqdm(total=len(df)) as pbar:
            for news in df["news"]:
                if isinstance(news, str):
                    ner_dict = extract_entities(news)
                else:
                    ner_dict = {"ner": "Missing"}
                output.append(ner_dict)
                pbar.update(1)

        ner_df = pd.DataFrame(output)
        ner_df["url"] = df["url"].tolist()
        ner_df.to_csv(country_dir + name, encoding="utf-8")

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 819/819 [00:50<00:00, 16.16it/s]
/var/folders/kn/jw0y9v615mjgg3_51g3s73zh0000gn/T/ipykernel_27371/1890609972.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["date"] = pd.to_datetime(df["date"], errors="coerce")
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 35489/35489 [41:52<00:00, 14.13it/s]


In [8]:

files = [file for file in os.listdir(target_dir) if "news" in file]


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 14484/14484 [11:54<00:00, 20.28it/s]


In [16]:
urls = []
for url, ner in zip(ner_df["url"], ner_df["ner"]):
    if "risk" in ner.lower():
        urls.append(url)

In [77]:
abc_dir = os.getcwd() + "/data/text/abc_au/"
abc_files = [file for file in os.listdir(abc_dir) if "news" in file]
test = pd.read_csv(abc_dir + abc_files[3]).drop("Unnamed: 0", axis=1)
test["tags"] = test["tags"].fillna("missing").str.lower()
tag_dict = {}
for tag in test.tags:
    tag_list = tag.split(",")[:-1]
    tag_list = [i.lower().strip() for i in tag_list]
    for i in tag_list:
        if i not in tag_dict.keys():
            tag_dict.update({str(i): 1})
        else:
            tag_dict[i] += 1

In [80]:
def get_news_count(data: pd.DataFrame, column: str):
    count_df = (data.set_index("date").groupby("ym")[[
        str(column)
    ]].count().reset_index().rename({str(column): str(column) + "_count"},
                                    axis=1))
    return count_df